# Semi-structured retrieval

We will test retrival of table information from the `Semi-structured Reports`dataset using various methods.

## Pre-requisites

In [ ]:
%pip install -U langchain langsmith langchainhub  langchain_benchmarks
%pip install --quiet chromadb openai "unstructured[all-docs]"

## Data Loading

### Get Public Dataset

Get the LangSmith public dataset for semi-structured data, `Semi-structured Reports`.

In [1]:
import os
from pathlib import Path

from langchain_benchmarks import clone_public_dataset, registry
from langchain_benchmarks.rag.tasks.semi_structured_reports import get_file_names

# Task
task = registry["Semi-structured Reports"]

# Files used
paths = list(get_file_names())
files = [str(p) for p in paths]

File /Users/rlm/miniforge3/envs/llama2/lib/python3.9/site-packages/langchain_benchmarks/rag/tasks/semi_structured_reports/indexing/chroma_db.zip does not exist. Downloading from GCS...
File https://storage.googleapis.com/benchmarks-artifacts/langchain-docs-benchmarking/semi_structured_earnings.zip downloaded.


### Base Case

Use PDF loader, which is naive to tables.

In [2]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


def load_and_split(file):
    """
    Load and split PDF files
    """

    loader = PyPDFLoader(file)
    pdf_pages = loader.load()

    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=2000, chunk_overlap=50
    )

    # Get chunks
    docs = text_splitter.split_documents(pdf_pages)
    texts = [d.page_content for d in docs]
    print(f"There are {len(texts)} text elements")
    return texts


baseline_texts = []
for fi in files:
    baseline_texts.extend(load_and_split(fi))

There are 11 text elements
There are 3 text elements


### Unstructured

Use tabble-aware splitting following cookbook [here](https://github.com/langchain-ai/langchain/blob/master/cookbook/Semi_Structured_RAG.ipynb).

In addition to the below pip packages, you will also need [poppler](https://pdf2image.readthedocs.io/en/latest/installation.html) and [tesseract](https://tesseract-ocr.github.io/tessdoc/Installation.html) in your system.

In [4]:
from unstructured.partition.pdf import partition_pdf


def categorize_elements(raw_pdf_elements):
    """
    Categorize extracted elements from a PDF into tables and texts.
    raw_pdf_elements: List of unstructured.documents.elements
    """
    tables = []
    texts = []
    for element in raw_pdf_elements:
        if "unstructured.documents.elements.Table" in str(type(element)):
            tables.append(str(element))
        elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
            texts.append(str(element))
    return texts, tables


def parse_pdf_unstructured(file):
    # Get elements
    unstructured_elements = partition_pdf(
        filename=file,
        extract_images_in_pdf=False,
        infer_table_structure=True,
        chunking_strategy="by_title",
        max_characters=4000,
        new_after_n_chars=3800,
        combine_text_under_n_chars=2000,
        image_output_dir_path=os.path.dirname(file),
    )

    # Categorize elements by type
    texts, tables = categorize_elements(unstructured_elements)
    print(f"There are {len(tables)} tables")
    print(f"There are {len(texts)} text elements")
    return texts, tables


unstructured_texts = []
unstructured_tables = []
for fi in files:
    texts, tables = parse_pdf_unstructured(fi)
    unstructured_texts.extend(texts)
    unstructured_tables.extend(tables)

# Optional: Enforce a specific token size for texts
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=4000, chunk_overlap=0
)
joined_texts = " ".join(unstructured_texts)
unstructured_texts_4k_token = text_splitter.split_text(joined_texts)

There are 6 tables
There are 12 text elements
There are 1 tables
There are 5 text elements


### Docugami

**-- TO-DO --**

Use table-aware splitting following cookbook [here](https://github.com/langchain-ai/langchain/blob/master/cookbook/docugami_xml_kg_rag.ipynb).

In [ ]:
! pip install docugami==0.0.4 dgml-utils==0.2.0

In [ ]:
from docugami_processing import upload_files, wait_for_xml

# Load
DOCSET_NAME = "Semi-Structured"
dg_docs = upload_files(files, DOCSET_NAME)
dgml_paths = wait_for_xml(dg_docs)

In [ ]:
def extract_docugami_file(dgml_path):
    with open(dgml_path, "r") as file:
        contents = file.read().encode("utf-8")
        chunks = get_chunks_str(
            contents,
            include_xml_tags=True,
            min_text_length=1024 * 8,  # 8k chars are ~2k tokens
            max_text_length=1024 * 8,  # 8k chars are ~2k tokens
        )

    # Tables
    tables = [c for c in chunks if "table" in c.structure.split()]
    print(f"There are {len(tables)} tables")

    # Text
    texts = [c for c in chunks if "table" not in c.structure.split()]
    print(f"There are {len(texts)} text elements")
    return texts, tables


docugami_texts = []
docugami_tables = []
for fname in files:
    # Get xml
    dgml_path = dgml_paths[fname]
    # Extract elelemtns
    texts, tables = extract_docugami_file(dgml_path)
    docugami_texts.extend(texts)
    docugami_tables.extend(tables)

## Indexing

### Base case

In [5]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

vectorstore_baseline = Chroma.from_texts(
    texts=baseline_texts, collection_name="baseline", embedding=OpenAIEmbeddings()
)

retriever_baseline = vectorstore_baseline.as_retriever()

### Multi-vector retriever w/ text

In [6]:
import uuid

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.schema.document import Document
from langchain.schema.output_parser import StrOutputParser
from langchain.storage import InMemoryStore


def generate_text_summaries(texts, tables, summarize_texts=False):
    """
    Summarize text elements
    texts: List of str
    tables: List of str
    summarize_texts: Bool to summarize texts
    """

    # Prompt
    prompt_text = """You are an assistant tasked with summarizing tables and text for retrieval. \
    These summaries will be embedded and used to retrieve the raw text or table elements. \
    Give a concise summary of the table or text that is well optimized for retrieval. Table or text: {element} """
    prompt = ChatPromptTemplate.from_template(prompt_text)

    # Text summary chain
    model = ChatOpenAI(temperature=0, model="gpt-4")
    summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()

    # Initialize empty summaries
    text_summaries = []
    table_summaries = []

    # Apply to text if texts are provided and summarization is requested
    if texts and summarize_texts:
        text_summaries = summarize_chain.batch(texts, {"max_concurrency": 5})
    elif texts:
        text_summaries = texts

    # Apply to tables if tables are provided
    if tables:
        table_summaries = summarize_chain.batch(tables, {"max_concurrency": 5})

    return text_summaries, table_summaries


def create_multi_vector_retriever(
    vectorstore,
    text_summaries=None,
    texts=None,
    table_summaries=None,
    tables=None,
    image_summaries=None,
    images=None,
):
    """
    Create retriever that indexes summaries, but returns raw images or texts
    """

    # Initialize the storage layer
    store = InMemoryStore()
    id_key = "doc_id"

    # Create the multi-vector retriever
    retriever = MultiVectorRetriever(
        vectorstore=vectorstore,
        docstore=store,
        id_key=id_key,
    )

    # Helper function to add documents to the vectorstore and docstore
    def add_documents(retriever, doc_summaries, doc_contents):
        doc_ids = [str(uuid.uuid4()) for _ in doc_contents]
        summary_docs = [
            Document(page_content=s, metadata={id_key: doc_ids[i]})
            for i, s in enumerate(doc_summaries)
        ]
        retriever.vectorstore.add_documents(summary_docs)
        retriever.docstore.mset(list(zip(doc_ids, doc_contents)))

    # Add texts, tables, and images
    if text_summaries:
        add_documents(retriever, text_summaries, texts)
    if table_summaries:
        add_documents(retriever, table_summaries, tables)
    if image_summaries:
        add_documents(retriever, image_summaries, images)

    return retriever

In [7]:
## Unstructured table extraction + MV Retriever

# Get text, table summaries
unstructured_text_summaries, unstructured_table_summaries = generate_text_summaries(
    unstructured_texts_4k_token, unstructured_tables, summarize_texts=False
)

# The vectorstore to use to index the summaries
unstructured_vectorstore = Chroma(
    collection_name="unstructured", embedding_function=OpenAIEmbeddings()
)

# Create retriever
retriever_unstructured = create_multi_vector_retriever(
    unstructured_vectorstore,
    unstructured_text_summaries,
    unstructured_texts_4k_token,
    unstructured_table_summaries,
    unstructured_tables,
)

In [ ]:
## Docugami table extraction + MV Retriever

# Get text, table summaries
docugami_text_summaries, docugami_table_summaries = generate_text_summaries(
    docugami_texts, docugami_tables, summarize_texts=False
)

# The vectorstore to use to index the summaries
docugami_vectorstore = Chroma(
    collection_name="docugami", embedding_function=OpenAIEmbeddings()
)

# Create retriever
retriever_docugami = create_multi_vector_retriever(
    docugami_vectorstore,
    docugami_text_summaries,
    docugami_texts,
    docugami_table_summaries,
    docugami_tables,
)

### Multi-vector retriever w/ images

Images extracted from the tables

In [24]:
import base64
import os

from langchain.schema.messages import HumanMessage


def encode_image(image_path):
    """Getting the base64 string"""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


def image_summarize(img_base64, prompt):
    """Make image summary"""
    chat = ChatOpenAI(model="gpt-4-vision-preview", max_tokens=1024)

    msg = chat.invoke(
        [
            HumanMessage(
                content=[
                    {"type": "text", "text": prompt},
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{img_base64}"},
                    },
                ]
            )
        ]
    )
    return msg.content


def generate_img_summaries(path):
    """
    Generate summaries and base64 encoded strings for images
    path: Path to list of .jpg files extracted by Unstructured
    """

    # Store base64 encoded images
    img_base64_list = []

    # Store image summaries
    image_summaries = []

    # Prompt
    prompt = """You are an assistant tasked with summarizing images for retrieval. \
    These summaries will be embedded and used to retrieve the raw image. \
    Give a concise summary of the image that is well optimized for retrieval."""

    # Apply to images
    for img_file in sorted(os.listdir(path)):
        if img_file.endswith(".jpg"):
            img_path = os.path.join(path, img_file)
            base64_image = encode_image(img_path)
            img_base64_list.append(base64_image)
            image_summaries.append(image_summarize(base64_image, prompt))

    return img_base64_list, image_summaries

In [25]:
# Temp: Store images from papers here
base_directory = "/Users/rlm/Desktop/semi_structured_reports/"
img_base64_list, image_summaries = generate_img_summaries(base_directory)

In [26]:
# The vectorstore to use to index the summaries
multi_modal_vectorstore = Chroma(
    collection_name="multi_modal", embedding_function=OpenAIEmbeddings()
)

retriever_multi_vector_img = create_multi_vector_retriever(
    multi_modal_vectorstore,
    image_summaries=image_summaries,
    images=img_base64_list,
)

## RAG

### Text

In [28]:
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough

def rag_chain(retriever):
    """
    RAG chain
    """

    # Prompt template
    template = """Answer the question based only on the following context, which can include text and tables:
    {context}
    Question: {question}
    """
    prompt = ChatPromptTemplate.from_template(template)

    # LLM
    model = ChatOpenAI(temperature=0, model="gpt-4")

    # RAG pipeline
    chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | model
        | StrOutputParser()
    )
    return chain


# Create RAG chains
chain_baseline = rag_chain(retriever_baseline)
chain_unstructured = rag_chain(retriever_unstructured)
# chain_docugami = rag_chain(retriever_docugami)

### Image

In [29]:
import io
import re

from IPython.display import HTML, display
from PIL import Image


def plt_img_base64(img_base64):
    """Disply base64 encoded string as image"""
    # Create an HTML img tag with the base64 string as the source
    image_html = f'<img src="data:image/jpeg;base64,{img_base64}" />'
    # Display the image by rendering the HTML
    display(HTML(image_html))


def looks_like_base64(sb):
    """Check if the string looks like base64"""
    return re.match("^[A-Za-z0-9+/]+[=]{0,2}$", sb) is not None


def is_image_data(b64data):
    """
    Check if the base64 data is an image by looking at the start of the data
    """
    image_signatures = {
        b"\xFF\xD8\xFF": "jpg",
        b"\x89\x50\x4E\x47\x0D\x0A\x1A\x0A": "png",
        b"\x47\x49\x46\x38": "gif",
        b"\x52\x49\x46\x46": "webp",
    }
    try:
        header = base64.b64decode(b64data)[:8]  # Decode and get the first 8 bytes
        for sig, format in image_signatures.items():
            if header.startswith(sig):
                return True
        return False
    except Exception:
        return False


def resize_base64_image(base64_string, size=(128, 128)):
    """
    Resize an image encoded as a Base64 string
    """
    # Decode the Base64 string
    img_data = base64.b64decode(base64_string)
    img = Image.open(io.BytesIO(img_data))

    # Resize the image
    resized_img = img.resize(size, Image.LANCZOS)

    # Save the resized image to a bytes buffer
    buffered = io.BytesIO()
    resized_img.save(buffered, format=img.format)

    # Encode the resized image to Base64
    return base64.b64encode(buffered.getvalue()).decode("utf-8")


def split_image_text_types(docs):
    """
    Split base64-encoded images and texts
    """
    b64_images = []
    texts = []
    for doc in docs:
        # Check if the document is of type Document and extract page_content if so
        if isinstance(doc, Document):
            doc = doc.page_content
        if looks_like_base64(doc) and is_image_data(doc):
            doc = resize_base64_image(doc, size=(1300, 600))
            b64_images.append(doc)
        else:
            texts.append(doc)
    return {"images": b64_images, "texts": texts}


def img_prompt_func(data_dict):
    """
    Join the context into a single string
    """
    formatted_texts = "\n".join(data_dict["context"]["texts"])
    messages = []

    # Adding image(s) to the messages if present
    if data_dict["context"]["images"]:
        for image in data_dict["context"]["images"]:
            image_message = {
                "type": "image_url",
                "image_url": {"url": f"data:image/jpeg;base64,{image}"},
            }
            messages.append(image_message)

    # Adding the text for analysis
    text_message = {
        "type": "text",
        "text": (
            "You are an analyst tasking with providing advice basded on images.\n"
            "You may be given a mix of text, tables, and image(s) usually of charts or graphs.\n"
            "Use this information to provide analysis related to the user question. \n"
            f"User-provided question: {data_dict['question']}\n\n"
            "Text and / or tables:\n"
            f"{formatted_texts}"
        ),
    }
    messages.append(text_message)
    return [HumanMessage(content=messages)]


def multi_modal_rag_chain(retriever):
    """
    Multi-modal RAG chain
    """

    # Multi-modal LLM
    model = ChatOpenAI(temperature=0, model="gpt-4-vision-preview", max_tokens=1024)

    # RAG pipeline
    chain = (
        {
            "context": retriever | RunnableLambda(split_image_text_types),
            "question": RunnablePassthrough(),
        }
        | RunnableLambda(img_prompt_func)
        | model
        | StrOutputParser()
    )

    return chain


chain_multimodal = multi_modal_rag_chain(retriever_multi_vector_img)

## Check

In [ ]:
# Check retrieval
query = "Analyzing the operating expenses for Q3 2023, which category saw the largest increase when compared to Q3 2022?"
docs = retriever_multi_vector_img.get_relevant_documents(query, limit=6)

# Check that we get back relevant images
plt_img_base64(docs[0])

# Eval

See guide [here](https://github.com/langchain-ai/langchain-benchmarks/blob/main/docs/source/notebooks/retrieval/semi_structured.ipynb).

In [19]:
clone_public_dataset(task.dataset_id, 
                     dataset_name=task.name)

  0%|          | 0/5 [00:00<?, ?it/s]

Finished fetching examples. Creating dataset...
New dataset created you can access it at https://smith.langchain.com/o/1fa8b1f4-fcb9-4072-9aa9-983e35ad61b8/datasets/01a8ff52-a089-43a8-aef4-281342846932.
Done creating dataset.


In [33]:
from functools import partial

from langsmith.client import Client

from langchain_benchmarks.rag import get_eval_config


def run_eval(chain, eval_run_name):
    """
    Run eval
    """
    client = Client()
    test_run = client.run_on_dataset(
        dataset_name=task.name,
        llm_or_chain_factory=lambda: (lambda x: x["question"]) | chain,
        evaluation=get_eval_config(),
        verbose=True,
        project_name = eval_run_name
    )


# Experiments
chain_map = {
    "baseline": chain_baseline,
    "unstructured": chain_unstructured,
    "multi_modal": chain_multimodal,
    ### "docugami": chain_docugami,
}

run_id = str(uuid.uuid4())
for project_name, chain in chain_map.items():
    run_eval(chain, project_name+"_"+run_id)

View the evaluation results for project 'baseline_28a6a20a-513b-43c1-9a5b-94e3ac32bb4b' at:
https://smith.langchain.com/o/1fa8b1f4-fcb9-4072-9aa9-983e35ad61b8/projects/p/d436015f-5d4a-4f27-beab-61982245180e?eval=true

View all tests for Dataset Semi-structured Reports at:
https://smith.langchain.com/o/1fa8b1f4-fcb9-4072-9aa9-983e35ad61b8/datasets/01a8ff52-a089-43a8-aef4-281342846932
[------------------------------------------------->] 5/5
 Eval quantiles:
                               0.25       0.5      0.75      mean      mode
embedding_cosine_distance  0.013626  0.025187  0.032054  0.027096  0.003338
faithfulness               1.000000  1.000000  1.000000  1.000000  1.000000
score_string:accuracy      0.900000  1.000000  1.000000  0.920000  1.000000
execution_time             5.494180  5.494180  5.494180  5.494180  5.494180
View the evaluation results for project 'unstructured_28a6a20a-513b-43c1-9a5b-94e3ac32bb4b' at:
https://smith.langchain.com/o/1fa8b1f4-fcb9-4072-9aa9-983e35ad61

Error Type: BadRequestError, Message: Error code: 400 - {'error': {'message': "This model's maximum context length is 8192 tokens. However, your messages resulted in 9441 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}


[------------------------------------------------->] 5/5
 Eval quantiles:
                               0.25       0.5      0.75      mean      mode
embedding_cosine_distance  0.009986  0.019680  0.034680  0.024987  0.000000
score_string:accuracy      0.975000  1.000000  1.000000  0.975000  1.000000
faithfulness               1.000000  1.000000  1.000000  1.000000  1.000000
execution_time             5.287483  5.287483  5.287483  5.287483  5.287483
View the evaluation results for project 'multi_modal_28a6a20a-513b-43c1-9a5b-94e3ac32bb4b' at:
https://smith.langchain.com/o/1fa8b1f4-fcb9-4072-9aa9-983e35ad61b8/projects/p/fd6537d8-d520-4ebc-9938-a223d8f2d371?eval=true

View all tests for Dataset Semi-structured Reports at:
https://smith.langchain.com/o/1fa8b1f4-fcb9-4072-9aa9-983e35ad61b8/datasets/01a8ff52-a089-43a8-aef4-281342846932
[>                                                 ] 0/5

ERROR:langchain.schema.callbacks.tracers.langchain:Connection error caused failure to patch https://api.smith.langchain.com/runs/7433aa90-29f8-4a80-b904-dda17079af10  in LangSmith API. Please confirm your LANGCHAIN_ENDPOINT. ConnectionError(MaxRetryError("HTTPSConnectionPool(host='api.smith.langchain.com', port=443): Max retries exceeded with url: /runs/7433aa90-29f8-4a80-b904-dda17079af10 (Caused by ProtocolError('Connection aborted.', timeout('The write operation timed out')))"))


[--------->                                        ] 1/5

ERROR:langchain.schema.callbacks.tracers.langchain:Connection error caused failure to post https://api.smith.langchain.com/runs  in LangSmith API. Please confirm your LANGCHAIN_ENDPOINT. ConnectionError(MaxRetryError("HTTPSConnectionPool(host='api.smith.langchain.com', port=443): Max retries exceeded with url: /runs (Caused by ProtocolError('Connection aborted.', timeout('The write operation timed out')))"))


[----------------------------->                    ] 3/5

[--------------------------------------->          ] 4/5

[------------------------------------------------->] 5/5


 Eval quantiles:
                                 0.25         0.5        0.75        mean  \
embedding_cosine_distance    0.044022    0.061628    0.089034    0.060458   
score_string:accuracy        0.100000    0.100000    1.000000    0.460000   
execution_time             186.300906  186.300906  186.300906  186.300906   

                                 mode  
embedding_cosine_distance    0.014748  
score_string:accuracy        0.100000  
execution_time             186.300906  
